In [35]:
import pandas as pd
df = pd.read_csv("data/Text/001.csv")


In [36]:
df

,number,words,start_time,end_time
0,0,hello,0.40,1.10
1,1,my,1.10,1.60
2,2,name,1.60,1.80
3,3,is,1.80,1.80
4,4,Angelinis,1.80,2.90
...,...,...,...,...
907,907,can,473.70,474.10
908,908,give,474.10,474.50
909,909,you,474.50,474.80
910,910,is,474.80,475.60


確定所有的數值是int or float

In [37]:
df['start_time'] = pd.to_numeric(df['start_time'])
df['end_time'] = pd.to_numeric(df['end_time'])


抓出最後一個字的時間

In [38]:
last_word_time = df.tail(1).values[0,2]

將每十秒的timeframe的字組成list

In [39]:
all_sentences001 = []
for t in range(0,480):
    sentences = []
    for index, rows in df.loc[(df["start_time"]>=t) & (df["start_time"]<=(t+10))].iterrows():
        sentences.append(rows["words"])
    all_sentences001.append(" ".join(sentences))

In [40]:
for i in range(1):
    print("=",all_sentences001[i],"=")

= hello my name is Angelinis and  I am 21 years old I was born in Bogota Colombia and in the place I used =


## the SBERT

In [41]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from torch import embedding
model = SentenceTransformer('all-MiniLM-L6-v2')


def get_cosine_similarity(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]
# Two lists of sentences
for i in range(0,len(all_sentences001)-1):
    #Compute embedding for both lists
    embeddings1 = model.encode(all_sentences001[i], convert_to_tensor=True)
    embeddings2 = model.encode(all_sentences001[i+1], convert_to_tensor=True)

    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings1, embeddings2)
    # convert from PyTorch tensor to numpy array
    print(get_cosine_similarity(embeddings1, embeddings2))
######


0.9482838
0.8144995
0.98601556
0.88233733
0.9117321
0.87547493
0.8069012
0.64813983
0.9320267
0.79163826
0.8841871
0.9513539
0.8379276
0.9398347
0.92691046
0.9870994
0.9031306
0.8240187
0.8508369
0.78724015
0.9551784
0.78759396
0.84378266
0.8690597
0.97452855
0.97081476
0.97538686
0.8606863
0.89825696
0.97958064
0.9803824
0.8489533
0.86509454
0.54132223
0.8190703
0.8658678
0.96830803
0.7903678
0.8629867
0.7121433
0.9865939
0.8236573
0.95243585
0.9116342
0.62665
0.94617176
0.9724674
0.8434484
1.0
0.7297085
0.84631133
0.89877886
0.7921372
0.9976187
0.9630146
0.90134525
0.93958646
0.8902681
0.992371
0.9968934
0.89163065
0.9154493
0.93578
0.9712582
0.95807046
0.9027128
0.8993736
0.74630284
0.9080129
0.8805833
0.9622211
0.8737424
0.9089768
0.8134278
0.9331467
1.0
0.96080506
0.97757107
0.72362375
0.8039621
0.9657691
0.96637285
0.9306378
0.7910426
0.9795533
0.9357407
0.9412188
0.744015
0.9295281
0.8322691
0.9335833
0.8506999
0.99999994
0.8593014
0.98947084
0.92923105
0.81612045
0.5166989
0.96

## The Wu-Pulmer Similarity

In [42]:
S = all_sentences001[0].split()
print(S)

['hello', 'my', 'name', 'is', 'Angelinis', 'and', 'I', 'am', '21', 'years', 'old', 'I', 'was', 'born', 'in', 'Bogota', 'Colombia', 'and', 'in', 'the', 'place', 'I', 'used']


### 製作去掉stopwords 的time frame的文字

In [71]:
from nltk.corpus import stopwords
all_sentences001 = []
stops = set(stopwords.words('english'))
for t in range(0,480):
    sentences = []
    for index, rows in df.loc[(df["start_time"]>=t) & (df["start_time"]<=(t+10))].iterrows():
        if rows["words"].lower() not in stops:
            sentences.append(rows["words"].lower())
    all_sentences001.append(" ".join(sentences))

In [80]:
import nltk
#nltk.download('omw-1.4')
#nltk.download('wordnet')
#nltk.download('stopwords')
from nltk.corpus import wordnet 
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
print(stops)
all_sentences_wu_sum = []
for sentence_num in range(0,len(all_sentences001)): #現在第幾句
    wu_pulmer_sum = 0
    the_sentence = str(all_sentences001[sentence_num]).split()
    print(the_sentence)
    for words_num in range(0, (len(the_sentence)-1)): #標定第一個字
        if len(wordnet.synsets(the_sentence[words_num])) > 0 :
            syns1 = wordnet.synsets(the_sentence[words_num])
        else:
            continue
        for words_num2 in range((words_num+1), len(the_sentence)): #標定第二個字
            if len(wordnet.synsets(the_sentence[words_num2])) > 0 :
                syns2 = wordnet.synsets(the_sentence[words_num2])
                print(the_sentence[words_num], the_sentence[words_num2], syns1[0].wup_similarity(syns2[0]))
            else:
                continue
            wu_pulmer_sum += syns1[0].wup_similarity(syns2[0])
    all_sentences_wu_sum.append(wu_pulmer_sum)


{'now', 'here', 'each', 'few', 'ours', 'again', 'of', 'yours', 'm', 'for', 'whom', 'are', 'through', 'in', 'ourselves', 'he', 'yourselves', 'below', 'wouldn', 'not', "hadn't", 'do', 'there', 'shan', 'weren', 'our', 'being', 'any', "you'll", 'my', 'its', 'who', 'did', 'into', "weren't", 'haven', "you've", 'should', "didn't", 'ma', 'above', 'no', 'mightn', 'hadn', 'can', 'i', 'needn', 'o', 've', 'has', 'we', 'herself', 'nor', 'be', 'and', 'off', 'itself', "doesn't", "it's", 'his', 'wasn', 'where', "wasn't", 'same', 'this', 'myself', 'been', 'having', 'their', 'aren', 'isn', 're', 'couldn', 'ain', 'd', 'how', "mustn't", 'at', 'shouldn', 'an', 'll', 'a', 'your', 'her', 'about', 'were', 'on', "shouldn't", "couldn't", 't', 'was', 'before', 'the', "aren't", 'until', 'why', 'down', 'will', 'didn', 'too', 'other', 'further', 'hers', 'as', 'out', 'doesn', 'him', 'such', 'them', "should've", "wouldn't", "mightn't", 'to', 'it', 'mustn', 'between', 'is', 'just', 'these', 'by', 'me', 'over', 'then',

In [81]:
print(all_sentences_wu_sum)

[10.018184550227893, 9.913002803869674, 9.881677137418151, 9.881677137418151, 10.87088062399006, 8.20958575271067, 11.470351302949902, 9.643179876165473, 9.80021025586041, 14.316015649297382, 13.338459347669877, 11.864635132669187, 11.864635132669187, 12.245592204493127, 18.4448009077266, 17.127515329527707, 17.127515329527707, 11.223523165860627, 8.202166323528553, 6.165204429213718, 5.984787984633188, 5.984787984633188, 10.351025831753386, 8.504741895531373, 7.994384700267053, 6.6338804985863815, 11.48144926109322, 11.48144926109322, 11.398040796492802, 11.19419464264665, 8.153093586839718, 8.153093586839718, 5.7365735056137535, 4.112863881904129, 1.935640992993934, 4.066182089324504, 1.964390142021721, 1.964390142021721, 1.0484126984126985, 0.6555555555555556, 2.153002553002553, 3.404070929070929, 5.026282051282052, 8.575427350427352, 8.575427350427352, 5.944658119658121, 7.5350427350427385, 13.034024635495218, 14.481932283402863, 14.481932283402863, 9.11644499291558, 11.29092280268